In [1]:
from training.training import finetune
from datasets import load_dataset, load_from_disk, DatasetDict, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from filtering import compute_perplexity, filter_dataset
import numpy as np

/home/coder/.local/lib/python310-conda-cuda-torch21/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_

In [2]:
disk_path = "data/train_eval_test_split"

# code_exercises = load_dataset("jinaai/code_exercises")["train"].select(range(25000))
# train_test_split = code_exercises.train_test_split(test_size=0.1)
# train_eval_split = train_test_split["train"].train_test_split(test_size=1/9)

# train_dataset = train_eval_split["train"]
# eval_dataset = train_eval_split["test"]
# test_dataset = train_test_split["test"]

# DatasetDict({
#     "train": train_dataset,
#     "eval": eval_dataset,
#     "test": test_dataset,
# }).save_to_disk(disk_path)

train_eval_test_split = load_from_disk(disk_path)

train_eval_split = DatasetDict({
    "train": train_eval_test_split["train"],
    "test": train_eval_test_split["eval"]
})

test_dataset = train_eval_test_split["test"]

In [3]:
model_name = "Salesforce/codegen-350M-mono"
model = AutoModelForCausalLM.from_pretrained(model_name).cuda()
tokenizer = AutoTokenizer.from_pretrained(model_name)

if tokenizer.__dict__.get("pad_token") is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

Some weights of the model checkpoint at Salesforce/codegen-350M-mono were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a model trained on another task or with another architecture (e

In [4]:
raw_perplexity_dataset = test_dataset.map(compute_perplexity, fn_kwargs={"model": model, "tokenizer": tokenizer}, batched=True, batch_size=8)
raw_perplexity = np.mean(raw_perplexity_dataset["perplexity"])
print("Raw perplexity:", raw_perplexity)

Map: 100%|██████████| 2500/2500 [02:33<00:00, 16.28 examples/s]

Raw perplexity: 19471.048578253172


In [4]:
finetune_kwargs = {
    "per_device_train_batch_size": 16,
    "warmup_ratio": 0.1,
    "per_device_eval_batch_size": 16,
    "gradient_accumulation_steps": 8,
    "evaluation_strategy": "steps",
    "eval_steps": 40,
    "save_strategy": "steps",
    "save_steps": 40,
}

In [5]:
finetuned_model = finetune(
    model,
    tokenizer,
    train_eval_split,
    run_name = "full_dataset_finetuning",
    **finetune_kwargs,
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: antonii-belyshev. Use `wandb login --relogin` to force relogin


/home/coder/.local/lib/python310-conda-cuda-torch21/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss,Model Preparation Time
40,0.143700,0.140454,0.005400
80,0.134400,0.130064,0.005400
120,0.125400,0.127595,0.005400
160,0.109900,0.129537,0.005400
200,0.103600,0.127603,0.005400
240,0.116100,0.126124,0.005400
280,0.106400,0.123922,0.005400
320,0.067700,0.137439,0.005400
360,0.068300,0.136801,0.005400
400,0.062600,0.137668,0.005400


eval/loss,█▁▁▁▁▁▁▁▁▁▁▁
eval/model_preparation_time,▁▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,█▅▃▁▂▁▃▂▁▂▂▃
eval/samples_per_second,▁▃▆█▇█▆▇█▇▇▆
eval/steps_per_second,▁▁▅███▅███▅▅
train/epoch,▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇█
train/grad_norm,█▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,▁▁▁▂▂▃▄▄▅▆▇█████████████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
train/loss,██▇▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/loss,0.13607


In [6]:
raw_perplexity_dataset = test_dataset.map(compute_perplexity, fn_kwargs={"model": finetuned_model, "tokenizer": tokenizer}, batched=True, batch_size=8)
finetuned_perplexity = np.mean(raw_perplexity_dataset["perplexity"])
print("Test perplexity for model finetuned on the whole train dataset", finetuned_perplexity)

Map: 100%|██████████| 2500/2500 [02:33<00:00, 16.28 examples/s]

Test perplexity for model finetuned on the whole train dataset 151.07375120973586


In [7]:
finetuned_model.save_pretrained("finetuned_checkpoints/raw_dataset_finetuning")

In [8]:
model = AutoModelForCausalLM.from_pretrained(model_name).cuda()
train_dataset = train_eval_split["train"].map(compute_perplexity, fn_kwargs={"model": model, "tokenizer": tokenizer}, batched=True, batch_size=8)
eval_dataset = train_eval_split["test"]

perplexities = []
for part in ["low", "mid", "high"]:
    model = AutoModelForCausalLM.from_pretrained(model_name).cuda()
    dataset = DatasetDict({
        "train": filter_dataset(train_dataset, "perplexity", 0.5, part),
        "test": eval_dataset
    })

    name = f"{part}_perplexity_dataset_finetuning"
    finetuned_model = finetune(
        model,
        tokenizer,
        dataset,
        run_name = name,
        **finetune_kwargs
    )

    finetuned_perplexity_dataset = test_dataset.map(compute_perplexity, fn_kwargs={"model": finetuned_model, "tokenizer": tokenizer}, batched=True, batch_size=8)
    perplexities.append(np.mean(finetuned_perplexity_dataset["perplexity"]))

    finetuned_model.save_pretrained(f"finetuned_checkpoints/{name}")

Some weights of the model checkpoint at Salesforce/codegen-350M-mono were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a model trained on another task or with another architecture (e

/home/coder/.local/lib/python310-conda-cuda-torch21/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss,Model Preparation Time
40,0.114000,0.139366,0.002800
80,0.088400,0.135823,0.002800
120,0.081300,0.136481,0.002800
160,0.047700,0.151900,0.002800
200,0.047900,0.152450,0.002800
240,0.029900,0.201103,0.002800


eval/loss,█▁▁▁▁▁▁
eval/model_preparation_time,▁▁▁▁▁▁▁
eval/runtime,▁▅▄▅▃▄█
eval/samples_per_second,█▄▅▄▆▄▁
eval/steps_per_second,█▁▁▁▁▁▁
train/epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇██████
train/global_step,▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇██
train/grad_norm,███▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,▁▁▂▂▃▃▄▅▅▅███████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆
train/loss,███▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/loss,0.2011


Map: 100%|██████████| 2500/2500 [02:33<00:00, 16.23 examples/s]
Some weights of the model checkpoint at Salesforce/codegen-350M-mono were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of

...

Map: 100%|██████████| 2500/2500 [02:34<00:00, 16.21 examples/s]
Some weights of the model checkpoint at Salesforce/codegen-350M-mono were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of

/home/coder/.local/lib/python310-conda-cuda-torch21/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss,Model Preparation Time
40,0.140100,0.136030,0.003000
80,0.115000,0.130943,0.003000
120,0.131300,0.129906,0.003000
160,0.081300,0.148275,0.003000
200,0.074100,0.143403,0.003000
240,0.055300,0.186252,0.003000
280,0.048600,0.169341,0.003000


eval/loss,█▁▁▁▁▁▁▁
eval/model_preparation_time,▁▁▁▁▁▁▁▁
eval/runtime,█▄▃▆▁▅▃▂
eval/samples_per_second,▁▅▆▃█▄▆▇
eval/steps_per_second,▁██▁████
train/epoch,▁▁▁▁▂▂▂▂▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███████
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇██
train/grad_norm,███▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,▁▂▂▂▃▄▅▅▆▇█████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆▆
train/loss,█▇▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/loss,0.16934


Map: 100%|██████████| 2500/2500 [02:33<00:00, 16.23 examples/s]


In [9]:
for ppl, part in zip(perplexities, ["low", "middle", "high"]):
    print(f"Test perplexity for model finetuned on {part} perplexity samples:", ppl)

Test perplexity for model finetuned on low perplexity samples: 144.29739919719697
Test perplexity for model finetuned on middle perplexity samples: 166.25313509335518
Test perplexity for model finetuned on high perplexity samples: 209.86736042041778


In [13]:
input_ids = tokenizer.encode(train_eval_split["test"][0]["problem"], return_tensors="pt").to(finetuned_model.device)

generated_code = tokenizer.decode(finetuned_model.generate(input_ids, max_new_tokens=200)[0])
print(generated_code)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


def get_absolute_values(numbers):
    """
    Given a list of numbers, calculate the absolute value for each number and return a set of absolute values.

    Inputs:
    - numbers: a list of numbers

    Returns:
    - absolute_set: a set containing the absolute values of the input numbers
    """
    absolute_set = set()
    for num in numbers:
        absolute_value = abs(num)
        absolute_set.add(absolute_value)
    return absolute_set


<|endoftext|>
